Generate player votes database, containing votes for Serie A matchday and player.

Votes data is manually downloaded from https://www.fantacalcio.it/voti-fantacalcio-serie-a

Serie A calendar is loaded from another file, to add information not containing in votes files: home/away, team opponent.

In [1]:
import pandas as pd
import numpy as np

In [2]:
cal = np.array(pd.read_excel('fantacalcio/seriea_calendar.xlsx', header = None))

cal_df = pd.DataFrame(columns = ['matchday', 'team1', 'team2'])

matchday = 0

for i in range(cal.shape[0]):
    if(cal[i, 0][0].isnumeric()):
        matchday = matchday + 1
        continue
    
    teams = cal[i, 0].split('-')
    
    frame = pd.DataFrame([[matchday, teams[0], teams[1]]], columns = cal_df.columns)

    cal_df = pd.concat([cal_df, frame], ignore_index = True)
    

In [3]:
cal_df

,matchday,team1,team2
0,1,Bologna,Milan
1,1,Empoli,Verona
2,1,Frosinone,Napoli
3,1,Genoa,Fiorentina
4,1,Inter,Monza
...,...,...,...
375,38,Verona,Inter
376,38,Juventus,Monza
377,38,Lazio,Sassuolo
378,38,Milan,Salernitana


In [4]:
df = pd.DataFrame(columns = ['matchday', 'player', 'team', 'oppteam', 'home', 'vote', 'goals', 'assists', 'cards_malus', 'fantavote'])

LAST_MATCH = 38

for matchday in range(1, LAST_MATCH + 1):
    
    votes_file = 'fantacalcio/voti/Voti_Fantacalcio_Stagione_2023_24_Giornata_' + str(matchday) + '.xlsx'

    try:
        rx = np.array(pd.read_excel(votes_file, header = None))
    except:
        break

    read = 0
    for i in range(rx.shape[0]):
        if(rx[i, 0] == "Cod."):
            read = 1
            team = rx[i-1, 0];
            continue

        if(read):
            if(isinstance(rx[i, 0], int)):
                if((isinstance(rx[i, 3], float) or isinstance(rx[i, 3], int)) and rx[i, 1] != "ALL") :
                    player = rx[i, 2];
                    vote = float(rx[i, 3])
                    goals = rx[i, 4] + rx[i, 8] - rx[i, 5]
                    assists = rx[i, 12]
                    cards_malus = rx[i, 10] * 0.5 + rx[i, 11]
                    
                    oppteam = ''
                    home = 0
                    for j in range(cal_df.shape[0]):
                        if(cal_df['matchday'][j] == matchday):
                            if(cal_df['team1'][j] == team):
                                oppteam = cal_df['team2'][j] 
                                home = 1
                            elif(cal_df['team2'][j] == team):
                                oppteam = cal_df['team1'][j] 
                                home = 0

                    goals_gen = goals * 3;
                    if(goals < 0):
                        goals_gen = goals
                        
                    fantavote = vote + goals_gen + assists - cards_malus 
                    
                    frame = pd.DataFrame([[matchday, player, team, oppteam, home, vote, goals, assists, cards_malus, fantavote]], columns = df.columns)

                    df = pd.concat([df, frame], axis = 0, ignore_index = True)
            else:
                read = 0
                continue
    
    print('loaded votes for matchday ' + str(matchday))

    df

loaded votes for matchday 1
loaded votes for matchday 2
loaded votes for matchday 3
loaded votes for matchday 4


In [5]:
df

,matchday,player,team,oppteam,home,vote,goals,assists,cards_malus,fantavote
0,1,Musso,Atalanta,Sassuolo,0,6.5,0,0,0.0,6.5
1,1,Zappacosta,Atalanta,Sassuolo,0,6.5,0,0,0.0,6.5
2,1,Djimsiti,Atalanta,Sassuolo,0,6.0,0,0,0.0,6.0
3,1,Kolasinac,Atalanta,Sassuolo,0,6.5,0,0,0.0,6.5
4,1,Zortea,Atalanta,Sassuolo,0,7.0,1,0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...
1154,4,Serdar,Verona,Bologna,1,6.0,0,0,0.5,5.5
1155,4,Suslov,Verona,Bologna,1,6.0,0,0,0.5,5.5
1156,4,Bonazzoli,Verona,Bologna,1,6.0,0,0,0.0,6.0
1157,4,Djuric,Verona,Bologna,1,5.5,0,0,0.0,5.5


In [6]:
df.to_excel('mid_outputs/players_votes.xlsx')

Elaborate data for past seasons.

Do not repeat if data is already present.

In [13]:
# BACKUP CODE FOR SEASON 2122 and 2021 (different calendar file format than 2223 and 2324)

cal_df = pd.read_excel('fantacalcio/season2122/seriea_calendar.xlsx')

to_drop = list()

for i in range(cal_df.shape[0]):
    if(not isinstance(cal_df['matchday'][i], int)):
        to_drop.append(i)
    
cal_df = cal_df.drop(to_drop)

cal_df = cal_df.reset_index(drop = True)

cal_df

,matchday,team1,team2
0,1,Verona,Sassuolo
1,1,Inter,Genoa
2,1,Empoli,Lazio
3,1,Torino,Atalanta
4,1,Bologna,Salernitana
...,...,...,...
375,38,Spezia,Napoli
376,38,Sassuolo,Milan
377,38,Inter,Sampdoria
378,38,Salernitana,Udinese


In [14]:
cal = np.array(pd.read_excel('fantacalcio/season2223/seriea_calendar.xlsx', header = None))

cal_df = pd.DataFrame(columns = ['matchday', 'team1', 'team2'])

matchday = 0

for i in range(cal.shape[0]):
    if(cal[i, 0][0].isnumeric()):
        matchday = matchday + 1
        continue
    
    teams = cal[i, 0].split('-')
    
    frame = pd.DataFrame([[matchday, teams[0], teams[1]]], columns = cal_df.columns)

    cal_df = pd.concat([cal_df, frame], ignore_index = True)
    

In [15]:
df = pd.DataFrame(columns = ['matchday', 'player', 'team', 'oppteam', 'home', 'vote', 'goals', 'assists', 'cards_malus', 'fantavote'])

LAST_MATCH = 38

for matchday in range(1, LAST_MATCH + 1):
    
    votes_file = 'fantacalcio/season2223/voti/Voti_Fantacalcio_Stagione_2022_23_Giornata_' + str(matchday) + '.xlsx'




    rx = np.array(pd.read_excel(votes_file, header = None))

    read = 0
    for i in range(rx.shape[0]):
        if(rx[i, 0] == "Cod."):
            read = 1
            team = rx[i-1, 0];
            continue

        if(read):
            if(isinstance(rx[i, 0], int)):
                if((isinstance(rx[i, 3], float) or isinstance(rx[i, 3], int)) and rx[i, 1] != "ALL") :
                    player = rx[i, 2];
                    vote = float(rx[i, 3])
                    goals = rx[i, 4] + rx[i, 8] - rx[i, 5]
                    assists = rx[i, 12]
                    cards_malus = rx[i, 10] * 0.5 + rx[i, 11]
                    
                    oppteam = ''
                    home = 0
                    for j in range(cal_df.shape[0]):
                        if(cal_df['matchday'][j] == matchday):
                            if(cal_df['team1'][j] == team):
                                oppteam = cal_df['team2'][j] 
                                home = 1
                            elif(cal_df['team2'][j] == team):
                                oppteam = cal_df['team1'][j] 
                                home = 0

                    goals_gen = goals * 3;
                    if(goals < 0):
                        goals_gen = goals
                        
                    fantavote = vote + goals_gen + assists - cards_malus 
                    
                    frame = pd.DataFrame([[matchday, player, team, oppteam, home, vote, goals, assists, cards_malus, fantavote]], columns = df.columns)

                    df = pd.concat([df, frame], axis = 0, ignore_index = True)
            else:
                read = 0
                continue

    df

In [16]:
df.to_excel('mid_outputs/season2223/players_votes.xlsx')